In [1]:
import json

filename = "pandas_function_openai.json"

with open(filename, "r", encoding="utf-8") as json_file:
        # Load the JSON data from the file
        data = json.load(json_file)

In [2]:
filename = "parent_summary_dict.json"

with open(filename, "r", encoding="utf-8") as json_file:
        # Load the JSON data from the file
        parent_summary_dict = json.load(json_file)

In [3]:
import networkx as nx

pandas_graph = nx.DiGraph()

parent_names = [(data[d]['name'],{"url":data[d]['url'],"type":"parent_node","node_description":parent_summary_dict[data[d]['name']]}) for d in data]
pandas_graph.add_nodes_from(parent_names)

In [4]:
for parent in data:
    parent_name = data[parent]['name']
    for sub_level in data[parent]['functions']:
        for func in sub_level['function_definitions']:
            func_name = func['function_name']
            # parent_trail = parent.split(".")[0]
            # print(func['parameter_names_desc'])
            # print(func.keys())
            pandas_graph.add_nodes_from([(func_name,{'function_desc':func['function_text'],'function_url':func['function_url'],'trail':parent_name,"type":"function_node","function_name":func['function_name'],'function_calling':str(func['openai_function']),'parameter_names_desc':str(func['parameter_names_desc'])})])
            pandas_graph.add_edge(parent_name,func_name)
        # break
    # break

In [5]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import os
from chromadb.utils.batch_utils import create_batches
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv(),override=True)

True

In [6]:

docs = []
metadata = []

for node, attributes in pandas_graph.nodes(data=True):
    if attributes['type'] == 'function_node':
        docs.append(attributes['function_desc'])
        # metadata.append({"metadata":str(attributes)})
        metadata.append(attributes)

    elif attributes['type'] == 'parent_node':
        docs.append(attributes['node_description'])
        attributes['name'] = node
        # metadata.append({"metadata":str(attributes)})
        metadata.append(attributes)

In [7]:
# import pprint
# pp = pprint.PrettyPrinter(depth=4)
# pp.pprint(attributes)

In [8]:
len(docs),len(metadata)

(1851, 1851)

In [9]:
metadata[-1200]

{'function_desc': 'Return the dataframe interchange object implementing the interchange protocol.',
 'function_url': 'https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.__dataframe__.html#pandas.DataFrame.__dataframe__',
 'trail': 'DataFrame',
 'type': 'function_node',
 'function_name': 'pandas.DataFrame.__dataframe__',
 'function_calling': "{'name': 'pandas#DataFrame#__dataframe__', 'descriptions': 'Return the dataframe interchange object implementing the interchange protocol.', 'parameters': {'type': 'object', 'properties': {'nan_as_null': {'type': 'boolean', 'description': 'nan_as_null is DEPRECATED and has no effect. Please avoid using\\nit; it will be removed in a future release.\\n'}, 'allow_copy': {'type': 'boolean', 'description': ''}}, 'required': []}}",
 'parameter_names_desc': "[{'param_name': 'nan_as_null', 'param_type': 'bool, default False', 'param_desc': 'nan_as_null is DEPRECATED and has no effect. Please avoid using\\nit; it will be removed in a future relea

In [10]:
def build_database(docs, metadata, api_key):
    database_path = "PANDAS"
    collection_name = "pandas_docs"
    load_dotenv(override=True)
    emb_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=api_key, model_name="text-embedding-3-small"
    )

    client = chromadb.PersistentClient(path=database_path)
    pandas_collection = client.create_collection(
        name=collection_name, embedding_function=emb_fn
    )

    openbb_ids = [f"id{i}" for i in range(len(docs))]
    batches = create_batches(
        api=client, ids=openbb_ids, documents=docs, metadatas=metadata
    )
    for batch in batches:
        pandas_collection.add(ids=batch[0], documents=batch[3], metadatas=batch[2])
    return pandas_collection

In [11]:
build_database(docs,metadata,os.environ['OPENAI_API_KEY'])

Collection(name=pandas_docs)

In [14]:
def load_database(api_key):
    database_path = "PANDAS"
    collection_name = "pandas_docs"
    emb_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=api_key, model_name="text-embedding-3-small"
    )
    client = chromadb.PersistentClient(path=database_path)
    pandas_collection = client.get_collection(
        name=collection_name, embedding_function=emb_fn
    )
    return pandas_collection

In [15]:
pandas_collection = load_database(os.environ['OPENAI_API_KEY'])

In [43]:
def generate_pairs(list1, list2):
    pairs = []
    for l1 in list1:
        for l2 in list2:
            curr_trail = l1
            curr_trail += f"-->{l2}"
            pairs.append(curr_trail)
    return [pairs]


def generate_pairs_recursive(trail_list):
    if len(trail_list) == 1:
        return trail_list[0]
    curr_pairs = generate_pairs(trail_list[-2], trail_list[-1])
    modified_trail_list = trail_list[:-2] + curr_pairs
    return generate_pairs_recursive(modified_trail_list)


def get_trail_list_pairs(trail_list_pairs,metadata_name="trail"):
    if len(trail_list_pairs) == 1:
        trail_where_clause = {metadata_name: {"$eq": trail_list_pairs[0]}}
    elif len(trail_list_pairs) > 1:
        trail_where_clause = {"$or": [{metadata_name: {"$eq": t}} for t in trail_list_pairs]}
    return trail_where_clause

In [60]:
import dspy
class FirstSecondLevel(dspy.Signature):
    "You are given a list of keys and values separated by semicolon."
    "Based on the query, you have to output the key that is most relevant to the question separated by semicolon."
    "Be precise and output only the relevant key or keys from the provided keys only."
    "Don't include any other information"

    query = dspy.InputField(prefix="Query which you need to classify: ", format=str)
    keys_values = dspy.InputField(prefix="Keys and Values: ", format=str)
    output = dspy.OutputField(
        prefix="Relevant Key(s): ", format=str, desc="relevant keys separated by semicolon"
    )


In [63]:
import ast

emb_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.environ["OPENAI_API_KEY"], model_name="text-embedding-3-small"
)

llm = dspy.OpenAI()
dspy.settings.configure(lm=llm)
class PandasAgentChroma(dspy.Module):
    def __init__(self,collection):
        super(PandasAgentChroma, self).__init__()
        self.collection = collection
        self.firstSecondLevel = dspy.Predict(FirstSecondLevel)
    
    def __call__(self, *args, **kwargs):
        return super().__call__(*args, **kwargs)
    
    def forward(self,query:str):
        query_emb = emb_fn([query])[0]

        # Parent level querying
        parent_level = self.collection.query(
                query_embeddings=query_emb,
                where={
                    "type": {"$eq": "parent_node"},
                },
                n_results=3,
            )
        parent_level_str = ""
        for parent_level_metadata in parent_level['metadatas'][0]:
            parent_level_str += f"{parent_level_metadata['name']}: {parent_level_metadata['node_description']}\n\n"
        
        parent_level_answer = self.firstSecondLevel(
            query=query, keys_values=parent_level_str
        ).output
        print(parent_level_str,parent_level_answer)
        trail_list = [parent_level_answer.split(";")]
        trail_list_pairs = generate_pairs_recursive(trail_list)
        
        trail_where_clause = get_trail_list_pairs(trail_list_pairs)

        function_level = self.collection.query(
            query_embeddings=query_emb,
            where={
                "$and":[
                    trail_where_clause,
                    {"type": {"$eq": "function_node"}},
                ]
            },
            n_results=5,
        )

        function_level_str = ""
        for function_level_metadata in function_level['metadatas'][0]:
            function_level_str += f"{function_level_metadata['function_name']}: {function_level_metadata['function_desc']}\n\n"
        print(function_level_str)
        function_level_answer = self.firstSecondLevel(
            query=query, keys_values=function_level_str
        ).output
        function_list = generate_pairs_recursive([function_level_answer.split(";")])
        function_where_clause = get_trail_list_pairs(function_list,"function_name")
        print(function_where_clause)
        functions = self.collection.get(
            where = {
                "$and":[
                    function_where_clause,
                    {"type": {"$eq": "function_node"}},
                ]
            }
        )
        return functions


In [65]:
pd = PandasAgentChroma(pandas_collection)
ans = pd("How to pivot a dataframe")
ans

General functions: The functions provided cover a wide range of data manipulation tasks in pandas. These include unpivoting a DataFrame from wide to long format, creating pivot tables, binning values, merging DataFrames, converting categorical variables, reshaping data, computing cross tabulations, discretizing data based on quantiles, concatenating pandas objects, detecting missing or non-missing values, converting data types, working with datetime data, inferring frequencies, evaluating expressions, guessing datetime formats, generating deterministic integers, hashing data, and building DataFrames from compatible sources. These functions provide a comprehensive toolkit for data analysis and manipulation in pandas. 

DataFrame: The pandas library provides a wide range of functions for working with tabular data. These functions include serialization to various file formats such as CSV, Excel, JSON, Feather, parquet, ORC, SQL databases, and Stata dta format. Additionally, there are func

{'ids': ['id591'],
 'embeddings': None,
 'metadatas': [{'function_calling': "{'name': 'pandas#DataFrame#pivot_table', 'descriptions': 'Create a spreadsheet-style pivot table as a DataFrame.', 'parameters': {'type': 'object', 'properties': {'values': {'type': 'list-like or scalar, optional', 'description': 'Column or columns to aggregate.\\n'}, 'index': {'type': 'column, Grouper, array, or list of the previous', 'description': 'Keys to group by on the pivot table index. If a list is passed,\\nit can contain any of the other types (except list). If an array is\\npassed, it must be the same length as the data and will be used in\\nthe same manner as column values.\\n'}, 'columns': {'type': 'column, Grouper, array, or list of the previous', 'description': 'Keys to group by on the pivot table column. If a list is passed,\\nit can contain any of the other types (except list). If an array is\\npassed, it must be the same length as the data and will be used in\\nthe same manner as column value

In [67]:
ans['metadatas'][0]['function_calling']

"{'name': 'pandas#DataFrame#pivot_table', 'descriptions': 'Create a spreadsheet-style pivot table as a DataFrame.', 'parameters': {'type': 'object', 'properties': {'values': {'type': 'list-like or scalar, optional', 'description': 'Column or columns to aggregate.\\n'}, 'index': {'type': 'column, Grouper, array, or list of the previous', 'description': 'Keys to group by on the pivot table index. If a list is passed,\\nit can contain any of the other types (except list). If an array is\\npassed, it must be the same length as the data and will be used in\\nthe same manner as column values.\\n'}, 'columns': {'type': 'column, Grouper, array, or list of the previous', 'description': 'Keys to group by on the pivot table column. If a list is passed,\\nit can contain any of the other types (except list). If an array is\\npassed, it must be the same length as the data and will be used in\\nthe same manner as column values.\\n'}, 'aggfunc': {'type': 'dictionary', 'description': 'If a list of func